## 💼 "All-Weather" Risk-Parity Portfolio Construction

Setup

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
tickers = ['SPY', 'TLT', 'GLD', 'TIP', 'BIL']  # Equities, Fixed Income, Commodities, Inflation-protected, Cash
start_date = '2013-01-01'
end_date = '2025-01-01'
vol_lookback = 126  # Volatility Lookback Period Length